In [ ]:
# !pip install pyTelegramBotAPI

In [71]:
import telebot
from telebot import types

import sqlite3
# подключаем базу данных
conn = sqlite3.connect('meeting_reminder.db')

# курсор для работы с таблицами
cursor = conn.cursor()

try:
    # sql запрос для создания таблицы
    query = 'DROP TABLE "meeting_reminder"'
    # исполняем его –> ура, теперь у нас есть таблица, куда будем все сохранять!
    cursor.execute(query)
except:
    pass

try:
    # sql запрос для создания таблицы
    query = 'CREATE TABLE "meeting_reminder" ("ID" INTEGER UNIQUE, "user_id" INTEGER, "meet_time" TEXT, "meet_location" TEXT, "meet_name" TEXT, "meet_text_msg" TEXT, PRIMARY KEY ("ID"))'
    # исполняем его –> ура, теперь у нас есть таблица, куда будем все сохранять!
    cursor.execute(query)
except:
    pass


In [64]:
# подключим токен нашего бота (bot_token - это просто наш файлик (в который мы токен положили))
from bot_token import token
bot = telebot.TeleBot(token)

In [65]:
# напишем, что делать нашему боту при команде старт
@bot.message_handler(commands=['start']) #декоратор 
def send_keyboard(message, text="Привет, я просто напоминаю о твоих встречах"):
    keyboard = types.ReplyKeyboardMarkup(row_width=2)  # наша клавиатура
    itembtn1 = types.KeyboardButton('Добавить встречу') # создадим кнопку
    itembtn2 = types.KeyboardButton('Показать встречи на сегодня')
    itembtn3 = types.KeyboardButton("Отредактировать встречу")
    itembtn4 = types.KeyboardButton('Удалить встречу')
    keyboard.add(itembtn1, itembtn2) # добавим кнопки 1 и 2 на первый ряд
    keyboard.add(itembtn3, itembtn4) # добавим кнопки 3, 4, 5 на второй ряд
    # но если кнопок слишком много, они пойдут на след ряд автоматически

    # пришлем это все сообщением и запишем выбранный вариант
    msg = bot.send_message(message.from_user.id,
                     text=text, reply_markup=keyboard) #бот возращает юзеру клавиатуру с кнопками

    # отправим этот вариант в функцию, которая его обработает
    bot.register_next_step_handler(msg, callback_worker)
    

In [120]:
bot.polling(none_stop=True)

In [117]:
# эта добавляет строчку с планом в хранилище
def add_plan(msg):
    try:
        meet_time = msg.text.split(';')[0].strip()
        meet_location = msg.text.split(';')[1].strip()
        meet_name = msg.text.split(';')[2].strip()
    except:
#         raise Exception("Строка имеет неверный формат")
        send_keyboard(msg, "Строка имеет неверный формат. Чем еще могу помочь?")
        return None
    
    with sqlite3.connect('meeting_reminder.db') as con:
        cursor = con.cursor()
        cursor.execute('INSERT INTO meeting_reminder (user_id, meet_time, meet_location, meet_name, meet_text_msg) VALUES (?, ?, ?, ?, ?)',
                       (msg.from_user.id, meet_time, meet_location, meet_name, msg.text.strip()))
        con.commit()
    bot.send_message(msg.chat.id, 'Запомню :-)')
    send_keyboard(msg, "Чем еще могу помочь?")

    
    

In [116]:
def ddd():
    print(1)
    return None
    print(2)
ddd

<function __main__.ddd()>

In [78]:
# просто функция, которая делает нам красивые строки для отправки пользователю
def get_plans_string(tasks):
    tasks_str = []
    for val in list(enumerate(tasks)): # val=(0, (дело1))
#         tasks_str.append(str(val[0] + 1) + ') ' + val[1][0] + val[1][1] + val[1][2]  + '\n')
        tasks_str.append(f'{str(val[0] + 1)}) {val[1][0]} {val[1][1]} {val[1][2]} \n')
    return ''.join(tasks_str)

# отправляем пользователю его планы
def show_plans(msg):
    with sqlite3.connect('meeting_reminder.db') as con:
        cursor = con.cursor()
        cursor.execute('SELECT meet_time, meet_location, meet_name, meet_text_msg FROM meeting_reminder WHERE user_id=={}'.format(msg.from_user.id))
        tasks = get_plans_string(cursor.fetchall())
        bot.send_message(msg.chat.id, tasks)
        send_keyboard(msg, "Чем еще могу помочь?")
        
# get_plans_string(())        

In [109]:
conn = sqlite3.connect('meeting_reminder.db')

# курсор для работы с таблицами
cursor = conn.cursor()

# sql запрос для создания таблицы
query = 'SELECT ID, meet_time, meet_location, meet_name, meet_text_msg FROM meeting_reminder ORDER BY ID'
# исполняем его –> ура, теперь у нас есть таблица, куда будем все сохранять!
cursor.execute(query)
cursor.fetchall()



[(2, '9:30', 'dion skonkin', 'Daily', '9:30;dion skonkin;Daily'),
 (3, '14:00', 'dion dsel', 'OnBoarding', '14:00;dion dsel;OnBoarding'),
 (5, '5:00', 'asdf', 'asdfg', 'Напишите в чат новое дело'),
 (6, '20:30', 'dion pupu', 'tututu', '20:30;dion pupu; tututu')]

In [84]:
# выделяет одно дело, которое пользователь хочет удалить
def delete_one_plan(msg):
    markup = types.ReplyKeyboardMarkup(row_width=2) #вот тут не разбивает по две штуки в ряд, потому что ццикл for это убивает. Нужно сразу по два выдавать
    with sqlite3.connect('meeting_reminder.db') as con:
        cursor = con.cursor()
        # достаем все задачи пользователя
        cursor.execute('SELECT meet_text_msg FROM meeting_reminder WHERE user_id=={}'.format(msg.from_user.id))
        # достанем результат запроса
        tasks = cursor.fetchall()
        for i in range(0, len(tasks), 2):
            try:
                markup.add(types.KeyboardButton(tasks[i][0]), types.KeyboardButton(tasks[i+1][0]))
            except:
                markup.add(types.KeyboardButton(tasks[i][0]))
#         for value in tasks:
#             markup.add(types.KeyboardButton(value[0]))
        msg = bot.send_message(msg.from_user.id,
                               text = "Выбери одну встречу из списка для удаления",
                               reply_markup=markup)
        bot.register_next_step_handler(msg, modify_one_plan)
        
        
# удаляет это дело
def delete_one_plan_(msg):
    with sqlite3.connect('meeting_reminder.db') as con:
        cursor = con.cursor()
        cursor.execute('DELETE FROM meeting_reminder WHERE user_id==? AND meet_text_msg==?', (msg.from_user.id, msg.text))
        con.commit()
        bot.send_message(msg.chat.id, 'Ура, минус одна встреча!')
        send_keyboard(msg, "Чем еще могу помочь?")

In [119]:
# выделяет одно дело, которое пользователь хочет удалить
def modify_one_plan(msg):
    markup = types.ReplyKeyboardMarkup(row_width=2) #вот тут не разбивает по две штуки в ряд, потому что ццикл for это убивает. Нужно сразу по два выдавать
    with sqlite3.connect('meeting_reminder.db') as con:
        cursor = con.cursor()
        # достаем все задачи пользователя
        cursor.execute('SELECT meet_text_msg FROM meeting_reminder WHERE user_id=={}'.format(msg.from_user.id))
        # достанем результат запроса
        tasks = cursor.fetchall()
        for i in range(0, len(tasks), 2):
            try:
                markup.add(types.KeyboardButton(tasks[i][0]), types.KeyboardButton(tasks[i+1][0]))
            except:
                markup.add(types.KeyboardButton(tasks[i][0]))
#         for value in tasks:
#             markup.add(types.KeyboardButton(value[0]))
        msg = bot.send_message(msg.from_user.id,
                               text = "Выбери одну встречу из списка для редактирования",
                               reply_markup=markup)
        bot.register_next_step_handler(msg, modify_one_plan_on)

    
def modify_one_plan_on(msg):
    msg_old = msg.text
    while True:
        try:
            msg = bot.send_message(msg.chat.id, 'Напишите в чат новое дело')
            msg_new = msg.text
        
            meet_time = msg_new.split(';')[0].strip()
            meet_location = msg_new.split(';')[1].strip()
            meet_name = msg_new.split(';')[2].strip()
            break
        except:  
            send_keyboard(msg, "Строка имеет неверный формат. Попробуй еще раз")
            return None
        
    bot.register_next_step_handler(msg, modify_one_plan_, msg_old, msg_new, meet_time, meet_location, meet_name)
        
# удаляет это дело
def modify_one_plan_(msg, msg_old, msg_new, meet_time, meet_location, meet_name):
    
    with sqlite3.connect('meeting_reminder.db') as con:
        cursor = con.cursor()
        cursor.execute(f'UPDATE meeting_reminder SET meet_time == ?, meet_location == ?, meet_name == ?, meet_text_msg==? WHERE user_id==? AND meet_text_msg==?', \
                       (meet_time, meet_location, meet_name, msg_new, msg.from_user.id, msg_old))
        con.commit()
        bot.send_message(msg.chat.id, 'Встреча изменена')
        send_keyboard(msg, "Чем еще могу помочь?")

In [88]:
# привязываем функции к кнопкам на клавиатуре
def callback_worker(call):
    if call.text == "Добавить встречу":
        msg = bot.send_message(call.chat.id, 'Давайте добавим дело! Напишите его в чат')
        bot.register_next_step_handler(msg, add_plan)

    elif call.text == "Показать встречи на сегодня":
        try:
            show_plans(call)
        except:
            bot.send_message(call.chat.id, 'Здесь пусто. Можно пойти поесть :-)')
            send_keyboard(call, "Чем еще могу помочь?")
            raise

    elif call.text == "Удалить встречу":
        try:
            delete_one_plan(call)
        except:
            bot.send_message(call.chat.id, 'Здесь пусто. Можно отдыхать :-)')
            send_keyboard(call, "Чем еще могу помочь?")

    elif call.text == "Отредактировать встречу":
        try:
            modify_one_plan(call)
        except:
            bot.send_message(call.chat.id, 'Здесь пусто. Можно отдыхать :-)')
            send_keyboard(call, "Чем еще могу помочь?")
            
#     elif call.text == "Удалить все дела из списка":
#         try:
#             delete_all_plans(call)
#         except:
#             bot.send_message(call.chat.id, 'Здесь пусто. Можно отдыхать :-)')
#             send_keyboard(call, "Чем еще могу помочь?")

#     elif call.text == "Ваше настроение":
#         try:
#             mood(call)
#         except:
#             #bot.send_message(call.chat.id, 'Что-то пошло не так')
#             raise

    elif call.text == "Пока все!":
        bot.send_message(call.chat.id, 'Хорошего дня! Когда захотите продолжнить нажмите на команду /start')

In [53]:
# import logging

# from telegram.ext import Updater,CallbackQueryHandler,CommandHandler, CallbackContext
# from telegram import  ReplyKeyboardRemove,ParseMode, Update

# import telegramcalendar, telegramjcalendar
# import utils
# import messages

# # Go to botfather and create a bot and copy the token and paste it here in token
# # token = "" # token of the bot


# logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
#                     level=logging.INFO)

# logger = logging.getLogger(__name__)

# def start(update, context):
#     context.bot.send_message(
#         chat_id=update.message.chat_id,
#         text=messages.start_message.format(update.message.from_user.first_name),
#         parse_mode=ParseMode.HTML)

# # A simple command to display the calender
# def calendar_handler(update, context):
#     update.message.reply_text(text=messages.calendar_message,
#                     reply_markup=telegramcalendar.create_calendar())
    

# def jcalendar_handler(update: Update, context: CallbackContext) -> int:
#     update.message.reply_text(
#         text=messages.jcalendar_message,
#         reply_markup=telegramjcalendar.create_calendar()
#     )

# def inline_handler(update, context):
#     query = update.callback_query
#     (kind, _, _, _, _) = utils.separate_callback_data(query.data)
#     if kind == messages.CALENDAR_CALLBACK:
#         inline_calendar_handler(update, context)
#     elif kind == messages.JCALENDAR_CALLBACK:
#         inline_jcalendar_handler(update, context)


# def inline_calendar_handler(update, context):
#     selected,date = telegramcalendar.process_calendar_selection(update, context)
#     if selected:
#         context.bot.send_message(chat_id=update.callback_query.from_user.id,
#                         text=messages.calendar_response_message % (date.strftime("%d/%m/%Y")),
#                         reply_markup=ReplyKeyboardRemove())


# def inline_jcalendar_handler(update: Update, context: CallbackContext):
#     selected, date = telegramjcalendar.process_calendar_selection(context.bot, update)
#     if selected:
#         context.bot.send_message(chat_id=update.callback_query.from_user.id,
#                 text=messages.jcalendar_response_message % date,
#                 reply_markup=ReplyKeyboardRemove())
        
# updater = Updater(token,use_context=True)
# dp=updater.dispatcher

# dp.add_handler(CommandHandler("start",start))
# dp.add_handler(CommandHandler("calendar",calendar_handler))
# dp.add_handler(CommandHandler("jcalendar",jcalendar_handler))
# dp.add_handler(CallbackQueryHandler(inline_handler))

ModuleNotFoundError: No module named 'telegram'